In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader, random_split
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [ ]:
train_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std= [0.229, 0.224, 0.225],
    ),
])
val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std= [0.229, 0.224, 0.225],
    ),
])

In [ ]:
DATA_DIR = r"C:\Data\Coding Programs\Project GenReal Ai\training_set"
full_dataset = datasets.ImageFolder(DATA_DIR, transform=train_transforms)
val_size   = int(0.2 * len(full_dataset))
train_size = len(full_dataset) - val_size

train_ds, val_ds = random_split(full_dataset, [train_size, val_size])
val_ds.dataset.transform = val_transforms
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True,  num_workers=4)
val_loader   = DataLoader(val_ds,   batch_size=32, shuffle=False, num_workers=4)

print(f"Training Data: {train_size}, Validation Data: {val_size}")


Training Data: 6404, Validation Data: 1601


In [ ]:
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)
model = model.to(device)


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to C:\Users\Administrator/.cache\torch\hub\checkpoints\resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:22<00:00, 4.46MB/s]


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(),lr=1e-4,weight_decay=1e-2)
scheduler = optim.lr_scheduler.StepLR(optimizer,step_size=5,gamma=0.1)

In [ ]:
best_val_acc = 0.0
num_epochs    = 20

for epoch in range(1, num_epochs+1):
    model.train()
    running_loss, running_corrects, total_samples = 0.0, 0, 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device),labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        preds = outputs.argmax(dim=1)
        running_corrects += (preds == labels).sum().item()
        total_samples += inputs.size(0)

    epoch_loss = running_loss / total_samples
    epoch_acc  = running_corrects / total_samples

    model.eval()
    val_loss, val_corrects, val_samples = 0.0, 0, 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs,labels= inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            preds = outputs.argmax(dim=1)
            val_corrects += (preds == labels).sum().item()
            val_samples += inputs.size(0)

    val_loss_epoch = val_loss/val_samples
    val_acc_epoch  = val_corrects/val_samples

    print(f"Epoch {epoch}/{num_epochs}  "
          f"Train: loss={epoch_loss:.4f}, acc={epoch_acc:.4f}   "
          f"Val: loss={val_loss_epoch:.4f}, acc={val_acc_epoch:.4f}")
    if val_acc_epoch > best_val_acc:
        best_val_acc = val_acc_epoch
        torch.save(model.state_dict(),"cat_dog_model.pth")
        print("Saved new best model")
    scheduler.step()


Epoch 1/20  Train: loss=0.0943, acc=0.9736   Val:   loss=0.0316, acc=0.9894
  Saved new best model!
Epoch 2/20  Train: loss=0.0162, acc=0.9947   Val:   loss=0.0236, acc=0.9906
  Saved new best model!
Epoch 3/20  Train: loss=0.0108, acc=0.9969   Val:   loss=0.0321, acc=0.9869
Epoch 4/20  Train: loss=0.0140, acc=0.9970   Val:   loss=0.0280, acc=0.9875
Epoch 5/20  Train: loss=0.0135, acc=0.9973   Val:   loss=0.0252, acc=0.9881
Epoch 6/20  Train: loss=0.0044, acc=0.9991   Val:   loss=0.0243, acc=0.9875
Epoch 7/20  Train: loss=0.0025, acc=0.9992   Val:   loss=0.0307, acc=0.9863
Epoch 8/20  Train: loss=0.0021, acc=0.9994   Val:   loss=0.0222, acc=0.9906
Epoch 9/20  Train: loss=0.0018, acc=0.9995   Val:   loss=0.0240, acc=0.9894
Epoch 10/20  Train: loss=0.0012, acc=0.9998   Val:   loss=0.0233, acc=0.9900
Epoch 11/20  Train: loss=0.0011, acc=0.9998   Val:   loss=0.0241, acc=0.9894
Epoch 12/20  Train: loss=0.0012, acc=0.9998   Val:   loss=0.0262, acc=0.9894
Epoch 13/20  Train: loss=0.0013, acc=

In [ ]:
if val_acc_epoch > best_val_acc:
        best_val_acc = val_acc_epoch
        torch.save(model.state_dict(), "cat_dog_model.pth")
        print("Saved new best model!")